In [1]:
import sqlite3
import matplotlib.pyplot as plt
import argparse

class data_shows():

    def __init__(self):
        # self.up = user_part()
        self.conn = sqlite3.connect("weather.db")
        self.c = self.conn.cursor()

    def shows_all(self, p_id):
        # show all the data of place
        print("""\n\n######################### ALL WEATHER DETAIL ###########################""")
        for i in range(8):
            t = self.c.execute(f"""SELECT * FROM PLACES_WEATHER{i} WHERE ID = {p_id} """)
            all_data = t.fetchall()[0]
            print(f"\n\n\nDay {i}", "\nLOCAL TIME:", all_data[1], "\nSUMMARY:", all_data[2], "\nWEATHER:", all_data[3],
                  "\nMIN_TEMP:", all_data[4], "\nMIN_TEMP_TIME:", all_data[5], "\nMAX_TEMP:", all_data[6],
                  "\nMAX_TEMP_TIME:", all_data[7], "\nWIND_SPEED:", all_data[8], "\nWINDBEARING:", all_data[9],
                  "\nHUMIDITY:", all_data[10], "\nCLOUD_COVER:", all_data[11], "\nAIR_PRESSURE:", all_data[12],
                  "\nVISIBILITY:", all_data[13])

    def shows_weather(self, p_id):

        print("""\n\n######################### WEATHER ###########################""")
        # showing the summary of 7 days weather
        t = self.c.execute(f"""SELECT * FROM WEATHER_SUMMARY WHERE ID = {p_id} """)
        summary_for_week = t.fetchall()[0][1]
        print("\n\nFor the future 7 days: ", summary_for_week)

        # weather and weather summary for each day
        t = self.c.execute(f"""SELECT * FROM PLACES_SUMMARY WHERE ID = {p_id} """)
        day_summary = t.fetchall()[0]
        t1 = self.c.execute(f"""SELECT * FROM PLACES_WEATHER WHERE ID = {p_id} """)
        day_weather = t1.fetchall()[0]

        for i in range(1, len(day_weather)):
            print(f"\n\nDAY {i-1}\n    weather summary: ", day_summary[i], f"\n    weather: ", day_weather[i])

    def shows_temp(self, p_id):
        # max and min temp for each day
        print("""\n\n######################### TEMPERATURE ###########################""")
        t = self.c.execute(f"""SELECT * FROM PLACES_TEMP WHERE ID = {p_id} """)
        day_temp = t.fetchall()[0]
        for i in range(1, len(day_temp)):
            if i % 2 != 0:
                print(f"\n\nDay {int(i/2 - 0.5)} \nMin Temperature: ", day_temp[i])
            else:
                print(f"Max Temperature: ", day_temp[i])

    def shows_wind(self, p_id):
        # wind speed and direction for each day
        print("""\n\n######################### WIND ###########################""")
        t = self.c.execute(f"""SELECT * FROM PLACES_WIND WHERE ID = {p_id} """)
        day_temp = t.fetchall()[0]
        for i in range(1, len(day_temp)):
            if i % 2 != 0:
                print(f"\n\nDay {int(i/2 - 0.5)} \nWind Speed: ", day_temp[i])
            else:
                print(f"Direction : ", day_temp[i])

    def shows_humi(self, p_id):
        # humidity for each days
        print("""\n\n######################### HUMIDITY ###########################""")
        t = self.c.execute(f"""SELECT * FROM PLACES_HUMIDITY WHERE ID = {p_id} """)
        day_humi = t.fetchall()[0]

        for i in range(1, len(day_humi)):
            print(f"\n\nDAY {i-1}\n    Humidity: ", day_humi[i])

    def shows_visi(self, p_id):
        # visibility for each days
        print("""\n\n######################### VISIBILITY ###########################""")
        t = self.c.execute(f"""SELECT * FROM PLACES_VISIBILITY WHERE ID = {p_id} """)
        day_vis = t.fetchall()[0]

        for i in range(1, len(day_vis)):
            print(f"\n\nDAY {i-1}\n    Visibility: ", day_vis[i])

    def shows_pres(self, p_id):
        # air pressure for each days
        print("""\n\n######################### AIR PRESSURE ###########################""")
        t = self.c.execute(f"""SELECT * FROM PLACES_AIR_PRESSURE WHERE ID = {p_id} """)
        day_pres = t.fetchall()[0]

        for i in range(1, len(day_pres)):
            print(f"\n\nDAY {i-1}\n    Air Pressure: ", day_pres[i])

    def shows_cloud(self, p_id):
        # cloud cover for each days
        print("""\n\n######################### CLOUD COVER ###########################""")
        t = self.c.execute(f"""SELECT * FROM PLACES_CLOUD_COVER WHERE ID = {p_id} """)
        day_cloud = t.fetchall()[0]

        for i in range(1, len(day_cloud)):
            print(f"\n\nDAY {i-1}\n    Cloud Cover: ", day_cloud[i])

    def shows_cities(self, p_id):
        # the weather details for each cities in each days
        print("""\n\n######################### CITIES WEATHER ###########################""")
        for i in range(6):
            print(f'\n\n\n########## Day {i}')
            t = self.c.execute(f"""
                     WITH CITY_T AS
                     (SELECT CITY_NAME, CITY.WOEID, DISTANCE 
                     FROM CITY JOIN CITY_PLACES ON CITY.WOEID = CITY_PLACES.WOEID
                     WHERE CITY_PLACES.ID = {p_id})

                     SELECT * 
                     FROM CITIES_WEATHER{i} JOIN CITY_T ON CITIES_WEATHER{i}.WOEID = CITY_T.WOEID

                     """)

            city_weather = t.fetchall()
            for j in range(len(city_weather)):
                print(f"""
                
                        City Name: {city_weather[j][-3]}
                        City WOEID: {city_weather[j][0]}
                        Distance: {city_weather[j][-1]}
                        Local Time: {city_weather[j][1]}
                        Weather: {city_weather[j][2]}
                        Max Temperature: {city_weather[j][4]}
                        Min Temperature: {city_weather[j][3]}
                        AVG Temperature: {city_weather[j][5]}
                        Wind Speed: {city_weather[j][6]}
                        Wind Direction: {city_weather[j][7]}
                        Humidity: {city_weather[j][8]}
                        Air Pressure: {city_weather[j][9]}
                        visibility: {city_weather[j][10]}
                        Predictability: {city_weather[j][11]}
                """)

    def shows_ana(self, p_id):
        print("""\n\n######################### ANALYSIS ###########################""")
        diff_place = []  # store the difference of Min_Max temp of places
        diff_city = {}  # store the difference of Min_Max temp of cities

        t = self.c.execute(f"""
                 SELECT PLACE  
                 FROM TRAVEL
                 WHERE ID = {p_id}
                 """)
        place_name = t.fetchall()[0][0]

        # create list for each cities
        t = self.c.execute(f"""
                 SELECT WOEID  
                 FROM CITY_PLACES
                 WHERE ID = {p_id}
                 """)
        woeids = t.fetchall()
        for woeid in woeids:
            diff_city[woeid[0]] = diff_city.get(woeid[0], [])

        for i in range(6):
            # each days data
            t = self.c.execute(f"""
                 SELECT CITY_PLACES.WOEID, DAY{i}_MIN_TEMP, DAY{i}_MAX_TEMP 
                 FROM CITY_PLACES JOIN CITIES_TEMP ON CITY_PLACES.WOEID = CITIES_TEMP.WOEID
                 WHERE CITY_PLACES.ID = {p_id}
                 """)
            c_t = t.fetchall()
            for j in range(len(c_t)):
                # each cities temp diff
                diff_city[c_t[j][0]].append(c_t[j][2] - c_t[j][1])

            # place temp diff
            t1 = self.c.execute(f"""
                 SELECT DAY{i}_MIN_TEMP, DAY{i}_MAX_TEMP 
                 FROM PLACES_TEMP
                 WHERE ID = {p_id}
                 """)
            p_t = t.fetchall()[0]
            diff_place.append(p_t[1] - p_t[0])

        keys = list(diff_city.keys())
        vals = list(diff_city.values())
        for i in range(len(keys)):
            # each city

            # get city's name
            t2 = self.c.execute(f"""
                 SELECT CITY_NAME 
                 FROM CITY
                 WHERE WOEID = {keys[i]}
                 """)
            name = t2.fetchall()[0][0]

            # graph
            day = ['DAY0', 'DAY1', 'DAY2', 'DAY3', 'DAY4', 'DAY5']
            x = list(range(len(day)))
            plt.figure(i)
            plt.subplot(111)
            plt.title(f"Temperature Difference between {place_name} and {name}")

            plt.bar(x, diff_place, width=0.5, label=f"{place_name}", tick_label=day, fc='y')
            for xn in range(len(x)):
                x[xn] = x[xn] + 0.5
            plt.bar(x, vals[i], 0.5, label=f"{name}", color='g')

            count = 0
            for j in range(len(vals[i])):

                if vals[i][j] > diff_place[j]:
                    # city_diif > place_diff
                    count += 1

            plt.legend()
            plt.show()

            if count / len(diff_place) > 0.5:

                print(
                    f"\n\nTemparature change huge in city '{name}' than in this places '{place_name}', you'd better bring more clothes")
            else:

                print(
                    f"\n\nTemparature change less in city '{name}' than in this places '{place_name}', you could only bring your wallet XD")


class user_part():

    def __init__(self):

        self.conn = sqlite3.connect("weather.db")
        self.c = self.conn.cursor()
        self.ds = data_shows()
        print("##################### Thanks for using this product! #######################")

    def check_quit(self):
        print("Do you want to quit?   1.Y   2.N")

        quit_or_not = input("""Please input the number for the one you want:""")

        if quit_or_not == '1':
            print("\n\n\n         Thanks for using the BEST weather software                ")
            print("                      HAPPY VACATION                 ")
            print("\n\n\n                     Quit Successfully                ")
            return None
        elif quit_or_not == '2':
            self.choose_partern()
        else:
            print("\n##################### PLEASE INPUT THE CORRECT NUMBER##################### \n")
            self.check_quit()

    def choose_partern(self):
        # user choose the part they want to use
        try:
            print("""
                        Which information would you like to gain:
                        0. Quit
                        1. The Places suggest by us
                        2. The Analysis Report
                    """)
            part = int(input("""Please input the number for the one you want:"""))

        except ValueError:
            print("\n ##################### PLEASE INPUT THE CORRECT NUMBER! #####################\n")
            self.choose_partern()

        except:
            print("\n ##################### UNKNOWN ERROR! ##################### \n")
            self.choose_partern()

        else:
            if part == 1:
                self.suggestion()
            elif part == 2:
                self.ana_report()
            elif part == 0:
                print("\n\n\n         Thanks for using the BEST weather software                ")
                print("                      HAPPY VACATION                 ")
                print("\n\n\n                     Quit Successfully                ")
                return None
            else:
                print("\n##################### PLEASE INPUT THE CORRECT NUMBER##################### \n")
                self.choose_partern()

    def suggestion(self):
        # suggest the place should go
        place_id = []
        t = self.c.execute("""SELECT * FROM PLACES_WEATHER """)
        weather = t.fetchall()

        if len(weather) == 0:
            print("It seems no places to go LOL, now you have time to play games and save the money.")
            return None

        print("\n\n\nAccording to the weather and visibility \nThose are the places we suggest you to go:")

        for i in range(len(weather)):
            # choose the place could go
            if list(weather[i]).count('clear-day') >= 5:
                # reqirement 1 : sunny day more than 5  
                t1 = self.c.execute(f"""SELECT * FROM PLACES_VISIBILITY WHERE ID = {weather[i][0]} """)
                vis = list(t1.fetchall()[0])

                count = 0
                for j in range(1, len(vis) - 1):
                    # check the visibility > 6
                    if vis[j] >= 6:
                        count += 1

                if count >= 6:
                    # requirement 2: num of days that have good visibility should >= 6 
                    place_id.append(vis[0])

        i = 0
        if place_id != []:
            for p_id in place_id:
                i += 1
                t = self.c.execute(f"""SELECT * FROM TRAVEL WHERE ID = {p_id}""")
                place = t.fetchall()
                if place[0][2] == None:
                    print(i, ".", place[0][1], 'locate in', place[0][3])
                else:
                    print(i, ".", place[0][1], 'locate in', place[0][2], ',', place[0][3])
        else:
            print(
                "\n\n\n#########################  No suggestion Sorry!!! You'd better stay at home and play some games !!! ")
        self.choose_place(i, place_id)

    def choose_place(self, num, place_id):
        # user choose the place that he/she interested
        print("""\n\n\n                Which Place's detail do you want to know?""")
        try:
            d_num = int(
                input(f"""\n\nNote:From 0 - {num}, 0 means Quit.\nPlease input the number for the one you want:"""))

        except ValueError:
            print("\n ##################### PLEASE INPUT THE CORRECT NUMBER! #####################\n")

            self.choose_place(num, place_id)

        except:
            print("\n ##################### UNKNOWN ERROR! ##################### \n")
            self.choose_place(self, num, place_id)
        else:
            if d_num == 0:
                self.check_quit()
                return None
            elif d_num >= 1 and d_num <= num:
                self.shows_detail(place_id[d_num - 1])
            else:
                print("\n ##################### PLEASE INPUT THE CORRECT NUMBER! ##################### \n")
                self.choose_place(num, place_id)

    def shows_detail(self, p_id):

        print("""                 
                                    What information you want?
                        0. All
                        1. Weather
                        2. Temprature
                        3. Wind
                        4. Humility
                        5. Visibility
                        6. Pressure
                        7. Cloudcover
                        8. Weather of the cities next to the place
                        9. Analysis
                        10. Quit
                        
            Note : You could combine them by using comma(','), for example: 1,2 
                        """)

        c_num = []
        n_num = ()
        ip = input(f"""Please input the number for the one you want:""")
        if len(ip) >= 1:
            # more than 1 input
            for i in ip.split(','):
                if i.isdigit():
                    # check is digital?
                    if int(i) >= 0 and int(i) <= 10:
                        # in the range
                        c_num.append(int(i))

        if c_num == []:
            print("""##################### PLEASE INPUT CORRECTLY! #####################""")
            self.shows_detail(p_id)

        else:
            # manage the inputs
            c_num.sort()
            n_num = set(c_num)

        for i in n_num:
            if i == 0:
                self.ds.shows_all(p_id)

            elif i == 1:
                self.ds.shows_weather(p_id)

            elif i == 2:
                self.ds.shows_temp(p_id)

            elif i == 3:
                self.ds.shows_wind(p_id)

            elif i == 4:
                self.ds.shows_humi(p_id)

            elif i == 5:
                self.ds.shows_visi(p_id)

            elif i == 6:
                self.ds.shows_pres(p_id)

            elif i == 7:
                self.ds.shows_cloud(p_id)

            elif i == 8:
                self.ds.shows_cities(p_id)

            elif i == 9:
                self.ds.shows_ana(p_id)

            elif i == 10:
                self.check_quit()
                return None

        self.shows_detail(p_id)

    def ana_report(self):

        print("""\n\n\n####################################################################################################
####################################### ANALYSIS REPORT ############################################""")
        print("""\n\n\n\n                                           DIRECTORY
                            
                            0. Quit
                            1. All temperature difference between places and cities
                            2. Analysis about temperature relate to lattitude and longitude
                            3. Analysis about wind speed relate to lattitude and longitude""")

        r = report()
        c = input("""                 Please choose which report you want to read""").strip()

        if c == '0':
            self.check_quit()
            return None
        elif c == '1':
            r.temp_diff()
            self.ana_report()
        elif c == '2':
            r.temp_lati_longi()
            self.ana_report()
        elif c == '3':
            r.wind_lat_log()
            self.ana_report()
        else:
            print("""################################ Please input Correctly #################################""")
            self.ana_report()

class report():
    def __init__(self):

        self.conn = sqlite3.connect("weather.db")
        self.c = self.conn.cursor()

    def temp_diff(self):
        # showing the temperature difference between cities and places

        t = self.c.execute(f"""
                     SELECT ID  
                     FROM TRAVEL
                     """)
        p_ids = t.fetchall()  # get all the p_id

        rate = 0

        for pp_id in p_ids:
            # for all the places

            p_id = pp_id[0]  # place_id
            diff_place = []  # store the difference of Min_Max temp of places
            diff_city = {}  # store the difference of Min_Max temp of cities

            t = self.c.execute(f"""
                     SELECT PLACE  
                     FROM TRAVEL
                     WHERE ID = {p_id}
                     """)
            place_name = t.fetchall()[0][0]

            # create list for each cities
            t = self.c.execute(f"""
                     SELECT WOEID  
                     FROM CITY_PLACES
                     WHERE ID = {p_id}
                     """)
            woeids = t.fetchall()
            for woeid in woeids:
                diff_city[woeid[0]] = diff_city.get(woeid[0], [])

            for i in range(6):
                # each days data
                t = self.c.execute(f"""
                     SELECT CITY_PLACES.WOEID, DAY{i}_MIN_TEMP, DAY{i}_MAX_TEMP 
                     FROM CITY_PLACES JOIN CITIES_TEMP ON CITY_PLACES.WOEID = CITIES_TEMP.WOEID
                     WHERE CITY_PLACES.ID = {p_id}
                     """)
                c_t = t.fetchall()
                for j in range(len(c_t)):
                    # each cities
                    diff_city[c_t[j][0]].append(c_t[j][2] - c_t[j][1])

                # place
                t1 = self.c.execute(f"""
                     SELECT DAY{i}_MIN_TEMP, DAY{i}_MAX_TEMP 
                     FROM PLACES_TEMP
                     WHERE ID = {p_id}
                     """)
                p_t = t.fetchall()[0]
                diff_place.append(p_t[1] - p_t[0])

            keys = list(diff_city.keys())
            vals = list(diff_city.values())
            for i in range(len(keys)):
                # each city

                # get city's name
                t2 = self.c.execute(f"""
                     SELECT CITY_NAME 
                     FROM CITY
                     WHERE WOEID = {keys[i]}
                     """)
                name = t2.fetchall()[0][0]

                ############### graph for temp diff 

                print(f"""\n\n\n################### Graph for {place_name} and {name}""")

                day = ['DAY0', 'DAY1', 'DAY2', 'DAY3', 'DAY4', 'DAY5']
                x = list(range(len(day)))
                plt.figure(i)
                plt.subplot(111)
                plt.title(f"Temperature Difference between {place_name} and {name}")

                plt.bar(x, diff_place, width=0.5, label=f"{place_name}", tick_label=day, fc='y')
                for xn in range(len(x)):
                    x[xn] = x[xn] + 0.5
                plt.bar(x, vals[i], 0.5, label=f"{name}", color='g')
                plt.legend()
                plt.show()

                ######## for rate
                large = 0
                small = 0
                draw = 0

                for j in range(len(vals[i])):
                    if vals[i][j] > diff_place[j]:
                        # city_diif > place_diff
                        large += 1

                    elif vals[i][j] == diff_place[j]:
                        # city_diif = place_diff
                        draw += 1

                    else:
                        # city_diif < place_diff
                        small += 1

                ############ graph of rate

                x = [large / 6, small / 6, draw / 6]
                plt.figure(i)
                plt.subplot(111)
                plt.title(f"Rate of Temperature Difference between {place_name} and {name}")

                plt.pie(x, labels=["C > P", "C < P", "C = P"], colors=('b', 'g', 'r'))
                plt.legend()
                plt.show()

                if large / 6 > 0.5:
                    print(f"""It seems temperature changes huge in city '{name}' than in place '{place_name}'
{x[0]*100}% of days temperature changes larger in city '{name}' than in place '{place_name}'
{x[1]*100}% of days temperature changes less in city '{name}' than in place '{place_name}'
{x[2]*100}% of days temperature changes same in city '{name}' and in place '{place_name}'""")
                elif small / 6 > 0.5:
                    print(f"""It seems temperature changes less in city '{name}' than in place '{place_name}'
{x[0]*100}% of days temperature changes larger in city '{name}' than in place '{place_name}'
{x[1]*100}% of days temperature changes less in city '{name}' than in place '{place_name}'
{x[2]*100}% of days temperature changes same in city '{name}' and in place '{place_name}'""")
                elif draw / 6 > 0.5:
                    print(f"""It seems temperature difference are the same in city '{name}' and in place '{place_name}'
{x[0]*100}% of days temperature changes larger in city '{name}' than in place '{place_name}'
{x[1]*100}% of days temperature changes less in city '{name}' than in place '{place_name}'
{x[2]*100}% of days temperature changes same in city '{name}' and in place '{place_name}'""")
                else:
                    print(f"""It seems temperature difference are dynamic in city '{name}' and in place '{place_name}'
{x[0]*100}% of days temperature changes larger in city '{name}' than in place '{place_name}'
{x[1]*100}% of days temperature changes less in city '{name}' than in place '{place_name}'
{x[2]*100}% of days temperature changes same in city '{name}' and in place '{place_name}'""")

    def temp_lati_longi(self):

        # places data
        p_id = []
        p_lat = []
        p_log = []
        p_avg_diff_temp = []

        t = self.c.execute("""
                SELECT  LOCATION.ID, LATI, LONGI, 
                        DAY0_MAX_TEMP, DAY0_MIN_TEMP, 
                        DAY1_MAX_TEMP, DAY1_MIN_TEMP, 
                        DAY2_MAX_TEMP, DAY2_MIN_TEMP, 
                        DAY3_MAX_TEMP, DAY3_MIN_TEMP, 
                        DAY4_MAX_TEMP, DAY4_MIN_TEMP,
                        DAY5_MAX_TEMP, DAY5_MIN_TEMP,
                        DAY6_MAX_TEMP, DAY6_MIN_TEMP,
                        DAY7_MAX_TEMP, DAY7_MIN_TEMP 
                
                FROM LOCATION JOIN PLACES_TEMP
                ON LOCATION.ID = PLACES_TEMP.ID
                
                 """)

        p_temps = t.fetchall()
        for i in range(len(p_temps)):
            temp_diff = []
            p_id.append(p_temps[i][0])  # store id
            p_lat.append(p_temps[i][1])  # store lat
            p_log.append(p_temps[i][2])  # store log
            for j in range(3, len(p_temps[i])):
                # calculate average temp diff
                if j % 2 != 0:
                    temp_diff.append(p_temps[i][j] - p_temps[i][j + 1])  # temp diff
            p_avg_diff_temp.append((sum(temp_diff) / len(temp_diff)) * 20)

        plt.figure(1)
        plt.gcf().set_size_inches(18.5, 18.5)
        plt.subplot(111)
        plt.title(f"Temperature Difference Graph for Places")

        plt.scatter(p_lat, p_log, s=p_avg_diff_temp, alpha=0.5, c=p_id)
        plt.show()

        # cities data

        woeid = []
        c_lat = []
        c_log = []
        c_avg_diff_temp = []

        t = self.c.execute("""
                SELECT  CITY.WOEID, LATT_LONG, 
                        DAY0_MAX_TEMP, DAY0_MIN_TEMP, 
                        DAY1_MAX_TEMP, DAY1_MIN_TEMP, 
                        DAY2_MAX_TEMP, DAY2_MIN_TEMP, 
                        DAY3_MAX_TEMP, DAY3_MIN_TEMP, 
                        DAY4_MAX_TEMP, DAY4_MIN_TEMP,
                        DAY5_MAX_TEMP, DAY5_MIN_TEMP

                FROM CITY JOIN CITIES_TEMP
                ON CITY.WOEID = CITIES_TEMP.WOEID
                 """)

        p_temps = t.fetchall()
        for i in range(len(p_temps)):
            temp_diff = []
            woeid.append(p_temps[i][0])  # store id
            lat, log = p_temps[i][1].split(',')
            c_lat.append(float(lat))  # store lat
            c_log.append(float(log))  # store log
            for j in range(2, len(p_temps[i])):
                # calculate average temp diff
                if j % 2 == 0:
                    temp_diff.append(p_temps[i][j] - p_temps[i][j + 1])  # temp diff
            c_avg_diff_temp.append((sum(temp_diff) / len(temp_diff)) * 20)

        plt.figure(2)
        plt.gcf().set_size_inches(18.5, 18.5)
        plt.subplot(111)
        plt.title(f"Temperature Difference Graph for Cities")

        plt.scatter(c_lat, c_log, s=c_avg_diff_temp, alpha=0.5, c=woeid)
        plt.show()

        # combine

        plt.figure(3)
        plt.gcf().set_size_inches(18.5, 18.5)
        plt.subplot(111)
        plt.title(f"Temperature Difference Graph for Combine")

        plt.scatter(c_lat, c_log, label='Cities', s=c_avg_diff_temp, alpha=0.5, c='r')
        plt.scatter(p_lat, p_log, label='Places', s=p_avg_diff_temp, alpha=0.5, c='b')
        plt.legend()
        plt.show()

        print("""######### Conclusion: It seems that the temperature difference are mostly large in the nature places than in the cites.
At the same time, it seems like the Places in the southwest get some huge temperature change, and cities which are in the middle have the huge change.""")

    def wind_lat_log(self):

        # places data
        p_id = []
        p_lat = []
        p_log = []
        p_avg_wind_speed = []

        t = self.c.execute("""
                SELECT  LOCATION.ID, LATI, LONGI, 
                        DAY0_WIND_SPEED, 
                        DAY1_WIND_SPEED,
                        DAY2_WIND_SPEED, 
                        DAY3_WIND_SPEED,
                        DAY4_WIND_SPEED,
                        DAY5_WIND_SPEED,
                        DAY6_WIND_SPEED,
                        DAY7_WIND_SPEED
                
                FROM LOCATION JOIN PLACES_WIND
                ON LOCATION.ID = PLACES_WIND.ID
                 """)

        p_temps = t.fetchall()
        for i in range(len(p_temps)):
            temp_ws = []
            p_id.append(p_temps[i][0])  # store id
            p_lat.append(p_temps[i][1])  # store lat
            p_log.append(p_temps[i][2])  # store log
            for j in range(3, len(p_temps[i])):
                # calculate average temp diff
                temp_ws.append(p_temps[i][j])  # temp diff
            p_avg_wind_speed.append((sum(temp_ws) / len(temp_ws)) * 20)

        plt.figure(1)
        plt.gcf().set_size_inches(18.5, 18.5)
        plt.subplot(111)
        plt.title(f"Average Wind Speed Graph for Places")

        plt.scatter(p_lat, p_log, s=p_avg_wind_speed, alpha=0.5, c=p_id)
        plt.show()

        # cities data

        woeid = []
        c_lat = []
        c_log = []
        c_avg_wind_speed = []

        t = self.c.execute("""
                SELECT  CITY.WOEID, LATT_LONG, 
                        DAY0_WIND_SPEED, 
                        DAY1_WIND_SPEED,
                        DAY2_WIND_SPEED, 
                        DAY3_WIND_SPEED,
                        DAY4_WIND_SPEED,
                        DAY5_WIND_SPEED

                FROM CITY JOIN CITIES_WIND
                ON CITY.WOEID = CITIES_WIND.WOEID
                 """)

        p_temps = t.fetchall()
        for i in range(len(p_temps)):
            temp_ws = []
            woeid.append(p_temps[i][0])  # store id
            lat, log = p_temps[i][1].split(',')
            c_lat.append(float(lat))  # store lat
            c_log.append(float(log))  # store log
            for j in range(2, len(p_temps[i])):
                # calculate average temp diff
                temp_ws.append(p_temps[i][j])  # wind speed diff
            c_avg_wind_speed.append((sum(temp_ws) / len(temp_ws)) * 20)

        plt.figure(2)
        plt.gcf().set_size_inches(18.5, 18.5)
        plt.subplot(111)
        plt.title(f"Average Wind Speed Graph for Cities")

        plt.scatter(c_lat, c_log, s=c_avg_wind_speed, alpha=0.5, c=woeid)
        plt.show()

        # combine

        plt.figure(3)
        plt.gcf().set_size_inches(18.5, 18.5)
        plt.subplot(111)
        plt.title(f"Average Wind Speed Graph for Combine")

        plt.scatter(c_lat, c_log, label='Cities', s=c_avg_wind_speed, alpha=0.5, c='r')
        plt.scatter(p_lat, p_log, label='Places', s=p_avg_wind_speed, alpha=0.5, c='b')
        plt.legend()
        plt.show()

        print(
            """########  Conclusion : It seems that the wind speed are larger on the around sides than in the middle.""")



In [2]:
if __name__ == '__main__':
   
    
        conn = sqlite3.connect("weather.db")
        c = conn.cursor()

        try:
            c.execute('''SELECT * FROM TRAVEL''')
            c.fetchall()
        except:
            print("""\n\n################## Error: No Database Detected #######################""")
            
            print("""\n\n################## Please download the database "weather.db" from github: https://github.com/xielidawan/Inf510_project.git #######################""")
           
            print("""\n\n################## System Closed #######################""")

        else:
            up = user_part()
            up.choose_partern()


##################### Thanks for using this product! #######################

                        Which information would you like to gain:
                        0. Quit
                        1. The Places suggest by us
                        2. The Analysis Report
                    
Please input the number for the one you want:0



         Thanks for using the BEST weather software                
                      HAPPY VACATION                 



                     Quit Successfully                
